In [1]:
pip install xgboost

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 3.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.cluster import KMeans
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score, cohen_kappa_score, davies_bouldin_score, calinski_harabasz_score, silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
# from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier

# import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/TS-S3-Ep5/train.csv'
file_key_2 = 'Tabular-Playground-Series/TS-S3-Ep5/test.csv'
file_key_3 = 'Tabular-Playground-Series/TS-S3-Ep5/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

## Enginering features
train['alcohol_density'] = train['alcohol'] * train['density']
train['sulphate/density'] = train['sulphates']  / train['density']
train['alcohol_sulphate'] = train['alcohol'] * train['sulphates']

test['alcohol_density'] = test['alcohol']  * test['density']
test['sulphate/density'] = test['sulphates']  / test['density']
test['alcohol_sulphate'] = test['alcohol'] * test['sulphates']

test_md = test.copy()

X = train[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates']]
Y = train['quality'] - 3

test_md = test_md[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates']]

# XGBoost Modeling like Crazy

In [2]:
XGB_cv_scores, XGB_imp = list(), list()
preds = list()

## Running 5 times CV
for i in range(10):
    
    print(i)
    ## Building RF model
    XGB_md = XGBClassifier(tree_method = 'hist',
                           colsample_bytree = 0.7, 
                           gamma = 5.5, 
                           learning_rate = 0.031, 
                           max_depth = 5, 
                           min_child_weight = 68, 
                           n_estimators = 8800, 
                           subsample = 0.41, 
                           random_state = i).fit(X, Y)
    XGB_imp.append(XGB_md.feature_importances_)
        
    ## Predicting on X_test and test
    XGB_pred_1 = XGB_md.predict(X)
    XGB_pred_2 = XGB_md.predict(test_md)
        
    ## Computing roc-auc score
    XGB_cv_scores.append(cohen_kappa_score(Y, XGB_pred_1, weights = 'quadratic'))
    preds.append(XGB_pred_2)

XGB_cv_score = np.mean(XGB_cv_scores)    
print('The average roc-auc score over 5-folds (run 5 times) is:', XGB_cv_score)

0
1
2
3
4
5
6
7
8
9
The average roc-auc score over 5-folds (run 5 times) is: 0.5227600573106826


In [21]:
preds

[array([2, 3, 2, ..., 2, 2, 2]),
 array([2, 3, 2, ..., 2, 2, 2]),
 array([2, 3, 2, ..., 2, 2, 2]),
 array([2, 3, 2, ..., 2, 2, 2]),
 array([2, 3, 2, ..., 2, 2, 2]),
 array([2, 3, 2, ..., 2, 2, 2]),
 array([2, 3, 2, ..., 2, 2, 2]),
 array([2, 3, 2, ..., 2, 2, 2]),
 array([2, 3, 2, ..., 2, 2, 2]),
 array([2, 3, 2, ..., 2, 2, 2])]

In [16]:
pd.DataFrame(preds)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,1322,1323,1324,1325,1326,1327,1328,1329,1330,1331,1332,1333,1334,1335,1336,1337,1338,1339,1340,1341,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363,1364,1365,1366,1367,1368,1369,1370,1371
0,2,3,2,3,3,3,3,3,3,3,2,3,3,3,3,2,3,2,2,4,4,3,3,3,2,2,4,3,4,2,2,3,2,2,3,2,4,2,2,2,2,3,2,4,4,3,2,2,3,2,...,3,4,2,2,2,2,2,2,2,2,2,4,4,2,2,3,2,3,3,2,4,2,2,3,4,3,2,3,3,2,2,3,2,2,2,2,2,3,4,4,3,2,2,4,3,3,3,2,2,2
1,2,3,2,3,3,3,3,3,3,3,2,3,3,3,3,2,3,2,2,4,4,3,3,3,2,2,4,3,4,2,2,3,2,2,3,2,4,2,2,2,2,3,2,4,4,3,2,2,3,2,...,3,4,2,2,2,2,2,2,3,3,2,4,4,2,2,3,2,3,3,2,4,2,3,3,4,3,2,3,3,2,2,3,2,2,2,2,2,3,3,4,3,2,3,4,3,3,3,2,2,2
2,2,3,2,3,3,3,3,3,3,3,2,3,3,3,3,2,3,2,2,4,4,3,3,3,2,2,4,3,4,2,2,3,2,2,3,2,4,2,2,2,2,3,2,4,4,3,2,2,3,2,...,3,3,2,2,2,2,2,2,3,2,2,4,4,2,2,3,2,3,3,2,4,2,3,3,4,3,2,3,3,2,2,3,2,2,2,2,2,3,4,4,3,2,3,4,3,3,3,2,2,2
3,2,3,2,3,2,3,3,3,3,3,2,3,3,3,3,2,3,2,2,4,4,3,3,3,2,2,4,3,4,2,2,3,2,2,3,2,4,2,2,2,2,3,2,4,4,3,2,2,3,2,...,3,3,2,2,2,2,2,2,2,2,2,4,4,2,2,3,2,3,3,2,4,2,2,2,4,3,2,3,3,2,2,3,2,2,2,2,2,3,3,4,3,2,2,4,3,3,3,2,2,2
4,2,3,2,3,3,3,3,3,3,3,2,3,3,3,3,2,3,2,2,4,4,3,3,3,2,2,4,3,4,2,2,3,2,2,3,2,4,2,2,2,2,3,2,4,4,3,2,2,3,2,...,3,3,2,2,2,2,2,2,2,2,2,4,4,2,2,3,2,3,3,2,4,2,2,3,4,3,2,3,3,2,2,3,2,2,2,2,2,3,4,4,3,2,2,4,3,3,3,2,2,2


In [3]:
XGB_preds_test = pd.DataFrame(preds).mode(axis = 0).loc[0, ] + 3

submission['quality'] = XGB_preds_test.astype(int)
submission.head()

,Id,quality
0,2056,5
1,2057,6
2,2058,5
3,2059,6
4,2060,6


In [4]:
submission['quality'].value_counts()

5    640
6    564
7    168
Name: quality, dtype: int64

In [5]:
submission.to_csv('XGB_baseline_full_FE_10.csv', index = False)